# EHR

*Classes*
   - Patient
     - CrCl Calculation Display
   - LabReference

In [60]:
!pip3 install sympy
!pip3 install pandas
!pip3 install tabulate
!pip3 install Jinja2

     |████████████████████████████████| 133 kB 6.0 MB/s eta 0:00:01


In [7]:
import sympy as sp
import random
from IPython.display import Markdown
import pandas as pd

## Patient (class)

   - CrCl (function)

In [8]:
class Patient:
   def __init__(self, name, weight, age, SerCreat):
      self.name = name
      self.id = random.randint(1000000,10000000)
      self.weight = weight
      self.age = age
      self.SerCreat = SerCreat
      self.CrCl_value = CrCl = ((140-self.age) * self.weight) / (72 * self.SerCreat)
   
   def as_dict (self):
      return {"name": self.name, "id": self.id, "weight":self.weight, "age":self.age, "SerumCreatinine": self.SerCreat, "CrCl": self.CrCl_value}
   
   @property
   def CrCl (self):
      name = self.name
      age = self.age 
      weight = self.weight
      scr = self.SerCreat
      
      a , w , Cr = sp.symbols("a_yr W_kg Cr_Ser")
      CrCl = ((140-a) * w) / (72 * Cr)
      
      md = Markdown("$\large CrCl_{" + name + "} = " + sp.Subs(CrCl,(a,w,Cr),(age,weight,scr))._repr_latex_()[1:]+ "  $\large = \\boxed {" + str(round(CrCl.subs({a:age,w:weight,Cr:scr}),1)) + "}$")
      
      return md

   def __repr__(self):
      return f'Name: {self.name}\n\tID:{self.id}\n\t{self.weight}kg\n\t{self.age}yr.of age\n\n\tCr(serum): {self.SerCreat}\n\t{self.CrCl_value}'

In [9]:
Patient("Josh Steinbecker",68,27, 0.888).CrCl

$\large CrCl_{Josh Steinbecker} = \displaystyle \left. \frac{W_{kg} \left(140 - a_{yr}\right)}{72 Cr_{Ser}} \right|_{\substack{ a_{yr}=27\\ W_{kg}=68\\ Cr_{Ser}=0.888 }}$  $\large = \boxed {120.2}$

In [13]:
Patient("Bob", 69, 59, 1.629).CrCl

$\large CrCl_{Bob} = \displaystyle \left. \frac{W_{kg} \left(140 - a_{yr}\right)}{72 Cr_{Ser}} \right|_{\substack{ a_{yr}=59\\ W_{kg}=69\\ Cr_{Ser}=1.629 }}$  $\large = \boxed {47.7}$

## LabReference (class)
```py
   >>> refID
   >>> Name
   >>> Low
   >>> High
   >>> Unit *
   >>> Class *
```

In [11]:
class LabReference:
   refID:      str
   name:       str
   lowbound:   float
   highbound:  float
   unit:       str
   def __init__(self,refID,name,low,high,Unit=None,Class="Chem"):
      self.refID = refID
      self.Class = Class
      self.name = name
      self.lowbound = low
      self.highbound = high
      self.unit = Unit
   def __repr__(self):
      return f'[{self.name} ({self.lowbound} - {self.highbound})]'
   def as_dict (self):
      return {"id": self.refID, "class":self.Class,"name":self.name, "low": self.lowbound, "high": self.highbound, "unit":self.unit }

In [33]:
NA = LabReference("$Na^+$","Sodium (serum)",136,142,"mg/dL")
K = LabReference("$K^+$","Potassium (serum)",3.5,5.1,"mg/dL")
CA = LabReference("$Ca^{2+}$","Calcium (serum)",8.1,10,"mg/dL")
HCO3 = LabReference("$CO_2^-$","Bicarbonate",21,28,"mg/dL")
GLU = LabReference("$GLU$","Glucose",90,100,"mg/dL")
VANC_TROUGH = LabReference("$Vanco_{min}$","Vancomycin (serum)(trough)",11,20,"mg/dL","ABX")
PLT = LabReference("$PLT$","Platelet",150000,400000,"/mL","Hemo")

ALLLABS = [NA,K,CA,HCO3,VANC_TROUGH,GLU,PLT]

In [56]:
display(Markdown("### ***LAB REFERENCE TABLE***"))
df = pd.DataFrame([x.as_dict() for x in ALLLABS])
df = df.set_index(["class","id"]).sort_index()
display(df)

### ***LAB REFERENCE TABLE***

name       low      high   unit
class id                                                                  
ABX   $Vanco_{min}$  Vancomycin (serum)(trough)      11.0      20.0  mg/dL
Chem  $CO_2^-$                      Bicarbonate      21.0      28.0  mg/dL
      $Ca^{2+}$                 Calcium (serum)       8.1      10.0  mg/dL
      $GLU$                             Glucose      90.0     100.0  mg/dL
      $K^+$                   Potassium (serum)       3.5       5.1  mg/dL
      $Na^+$                     Sodium (serum)     136.0     142.0  mg/dL
Hemo  $PLT$                            Platelet  150000.0  400000.0    /mL

In [57]:
mdtab = lambda df: display(Markdown ("### ***LAB REFERENCE TABLE***"), Markdown(df.to_markdown()))
mdtab(df)

### ***LAB REFERENCE TABLE***

|                          | name                       |      low |     high | unit   |
|:-------------------------|:---------------------------|---------:|---------:|:-------|
| ('ABX', '$Vanco_{min}$') | Vancomycin (serum)(trough) |     11   |     20   | mg/dL  |
| ('Chem', '$CO_2^-$')     | Bicarbonate                |     21   |     28   | mg/dL  |
| ('Chem', '$Ca^{2+}$')    | Calcium (serum)            |      8.1 |     10   | mg/dL  |
| ('Chem', '$GLU$')        | Glucose                    |     90   |    100   | mg/dL  |
| ('Chem', '$K^+$')        | Potassium (serum)          |      3.5 |      5.1 | mg/dL  |
| ('Chem', '$Na^+$')       | Sodium (serum)             |    136   |    142   | mg/dL  |
| ('Hemo', '$PLT$')        | Platelet                   | 150000   | 400000   | /mL    |

In [54]:
df[df.id == "$PLT$"]

,id,class,name,low,high,unit
6,$PLT$,Hemo,Platelet,150000.0,400000.0,/mL


In [64]:
df.style.format({"low":"{:,g}","high":"{:,g}"})